# CAI - Colony Avalanche Ecosystem Index

Crypto index tracking is a popular passive investment strategy. Since an index cannot be traded directly, index tracking refers to the process of creating a set of crypto assets managed by smart contract that approximates its performance. A straightforward way to do that is to purchase all the assets that compose an index in appropriate quantities.

In order to compensate for the price changes of the individual assets in the index we need to rebalance the portfolio. Choosing the optimal frequency is not an easy task and many factors (e.g., transaction costs, volatility, type of assets, etc.) need to be taken into account.

Conventional approaches to index portfolio rebalancing include periodic and tolerance band rebalancing. With periodic rebalancing, the portfolio is adjusted from its current weights back to the target weights at a consistent time interval (e.g., monthly or quarterly).

## Setting

In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots 

In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [5]:
from model import Engine, Index, VTokens, Trade

In [6]:
import plotting

## Assets filtering 

### Token inclusion cretira

1. **Project and token characteristics**
    1. The project's token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project's token must not have the ability to pause token transfers.
    1. The project's protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of onchain liquidity on a single pair.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available.



### Tokens list

In [7]:
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

In [8]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [9]:
avalanche_tokens = {} 
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'avalanche' == list(coin['platforms'].keys())[0]:
        avalanche_tokens[coin['id']] = coin

In [10]:
tokens_data = pd.DataFrame(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc'))
for i in range(2, 20):
    tokens_data = tokens_data.append(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
tokens_data = tokens_data.reset_index(drop=True)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_25997/4237399937.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_25997/4237399937.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_25997/4237399937.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_25997/4237399937.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_25997/4237399937.py:3: FutureWarning:

The frame.append method is depreca

In [11]:
tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,31638.000000,602803380915,1,6.643540e+11,3.142179e+10,31879.000000,...,2.100000e+07,2.100000e+07,69045.000000,-54.23273,2021-11-10T14:24:11.849Z,67.810000,46501.34216,2013-07-06T00:00:00.000Z,None,2022-05-31T10:34:32.381Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1971.990000,238663432489,2,NaN,1.769765e+10,2007.880000,...,NaN,NaN,4878.260000,-59.59236,2021-11-10T14:24:19.604Z,0.432979,455162.37441,2015-10-20T00:00:00.000Z,"{'times': 82.31222760314009, 'currency': 'btc'...",2022-05-31T10:34:56.952Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.000000,72558962162,3,NaN,5.599252e+10,1.004000,...,7.253725e+10,NaN,1.320000,-24.47456,2018-07-24T00:00:00.000Z,0.572521,74.53920,2015-03-02T00:00:00.000Z,None,2022-05-31T10:32:22.794Z
3,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.002000,54036125244,4,NaN,5.836717e+09,1.006000,...,5.390743e+10,NaN,1.170000,-14.61819,2019-05-08T00:40:28.300Z,0.891848,12.27000,2021-05-19T13:14:05.611Z,None,2022-05-31T10:35:38.463Z
4,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,319.980000,53806599766,5,5.380660e+10,1.261946e+09,323.510000,...,1.681370e+08,1.681370e+08,686.310000,-53.45517,2021-05-10T07:24:17.097Z,0.039818,802156.89602,2017-10-19T00:00:00.000Z,None,2022-05-31T10:35:03.305Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,razor-network,razor,Razor Network,https://assets.coingecko.com/coins/images/1379...,0.007141,1341321,1896,7.144747e+06,1.702530e+05,0.007172,...,1.000000e+09,1.000000e+09,0.978539,-99.26890,2021-02-04T13:52:01.426Z,0.005252,36.21772,2022-05-20T16:23:46.596Z,None,2022-05-31T10:36:01.051Z
1896,katana-inu,kata,Katana Inu,https://assets.coingecko.com/coins/images/2187...,0.000289,1341066,1897,1.445500e+07,1.632360e+05,0.000296,...,5.000000e+10,5.000000e+10,0.009098,-96.82838,2021-12-24T01:38:28.043Z,0.000199,45.09724,2022-05-12T07:29:43.106Z,None,2022-05-31T10:36:43.804Z
1897,arcticcoin,arc,Advanced Technology Coin,https://assets.coingecko.com/coins/images/633/...,0.044531,1328617,1898,NaN,1.169000e+01,NaN,...,6.000000e+07,NaN,6.440000,-99.30852,2017-06-10T00:00:00.000Z,0.000199,22234.27196,2021-04-15T15:41:14.410Z,None,2022-05-26T02:01:28.582Z
1898,zodium,zodi,Zodium,https://assets.coingecko.com/coins/images/2118...,0.019336,1327151,1899,NaN,8.097100e+04,0.020107,...,8.888889e+08,NaN,0.731751,-97.35728,2021-12-31T02:34:09.727Z,0.016507,17.15179,2022-05-30T11:17:55.338Z,None,2022-05-31T10:35:32.532Z


In [12]:
avalanche_tokens_data = tokens_data[tokens_data['id'].isin(list(avalanche_tokens.keys()))]

In [13]:
avalanche_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
14,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,26.910000,7310362250,15,1.939465e+10,9.585964e+08,28.890000,...,4.048230e+08,7.200000e+08,144.960000,-81.48437,2021-11-21T14:18:56.538Z,2.800000,858.22440,2020-12-31T13:15:21.540Z,None,2022-05-31T10:35:35.201Z
242,yusd-stablecoin,yusd,YUSD Stablecoin,https://assets.coingecko.com/coins/images/2502...,0.995103,131973451,243,NaN,1.059603e+06,0.999958,...,1.326117e+08,NaN,1.025000,-2.87376,2022-04-19T18:17:51.425Z,0.942459,5.58827,2022-05-11T12:54:22.300Z,None,2022-05-31T10:35:50.115Z
304,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.357717,97562173,305,1.790379e+08,6.483083e+06,0.381958,...,3.665385e+08,5.000000e+08,5.090000,-92.97671,2021-11-21T14:27:00.202Z,0.026588,1243.76436,2021-08-11T14:01:35.223Z,None,2022-05-31T10:36:51.348Z
330,benqi-liquid-staked-avax,savax,BENQI Liquid Staked AVAX,https://assets.coingecko.com/coins/images/2365...,27.250000,83547089,331,NaN,3.269280e+05,29.170000,...,3.069319e+06,NaN,103.550000,-73.78981,2022-04-02T13:44:54.612Z,21.730000,24.87437,2022-05-27T19:24:15.546Z,None,2022-05-31T10:34:51.620Z
411,step-app-fitfi,fitfi,Step App,https://assets.coingecko.com/coins/images/2501...,0.140605,57698071,412,7.036350e+08,5.463075e+07,0.153330,...,5.000000e+09,5.000000e+09,0.731881,-80.73425,2022-05-05T10:49:00.713Z,0.081387,73.25000,2022-05-12T07:19:29.374Z,None,2022-05-31T10:36:13.152Z
472,wonderland,time,Wonderland,https://assets.coingecko.com/coins/images/1812...,50.040000,45365042,473,4.821136e+07,2.098220e+05,55.800000,...,1.216453e+07,9.567396e+05,10063.720000,-99.50015,2021-11-07T19:15:45.691Z,48.950000,2.76404,2022-05-31T09:44:40.055Z,None,2022-05-31T10:34:51.041Z
545,snowbank,sb,Snowbank,https://assets.coingecko.com/coins/images/1994...,220.240000,35114214,546,NaN,5.042700e+03,225.470000,...,1.596400e+05,NaN,8356.800000,-97.37463,2021-11-08T05:30:08.317Z,138.860000,57.99722,2022-01-23T06:18:28.046Z,None,2022-05-31T10:35:48.468Z
563,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.014980,32907354,564,1.076677e+08,4.878122e+06,0.015734,...,7.200000e+09,7.200000e+09,0.394170,-96.21677,2021-08-24T03:58:11.390Z,0.008900,67.55289,2022-05-12T07:26:36.407Z,None,2022-05-31T10:37:06.593Z
680,avalaunch,xava,Avalaunch,https://assets.coingecko.com/coins/images/1546...,0.904038,22467804,681,9.017452e+07,1.688720e+06,0.963063,...,1.000000e+08,1.000000e+08,20.090000,-95.52361,2021-12-03T14:12:24.965Z,0.471748,90.60256,2021-07-21T05:06:03.647Z,None,2022-05-31T10:37:05.428Z
910,platypus-finance,ptp,Platypus Finance,https://assets.coingecko.com/coins/images/2172...,0.294733,10624087,911,8.834757e+07,1.705690e+05,0.321236,...,3.000000e+08,3.000000e+08,16.690000,-98.23990,2022-01-16T07:08:24.929Z,0.224090,31.10045,2022-05-27T09:25:08.321Z,None,2022-05-31T10:34:56.052Z


### Retrieving marketcaps and prices

In [14]:
prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in avalanche_tokens_data[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
    
    df = pd.DataFrame(data['prices'], columns=['date', symbol])
    df = df[df[symbol] > 0]
    df['date'] = pd.to_datetime(df['date'], unit='ms').dt.date
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date', drop=True)
    df = df.loc[~df.index.duplicated(keep='first')]
    
    if len(df) < 180:
        print(f'Excluding {symbol}, prices data available only for {len(df)} < 180 days')
        continue
    prices_data = pd.concat([prices_data, df], axis=1)
    
    df = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df = df[df[symbol] > 0]
    df['date'] = pd.to_datetime(df['date'], unit='ms').dt.date
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date', drop=True)
    df = df.loc[~df.index.duplicated(keep='first')]
    
    if len(df) < 180:
        print(f'Note: {symbol}, marketcap data available only for {len(df)} < 180 days')
        ## continue
    marketcaps = pd.concat([marketcaps, df], axis=1)
    
    

Excluding YUSD, prices data available only for 45 < 180 days
Excluding SAVAX, prices data available only for 102 < 180 days
Excluding FITFI, prices data available only for 36 < 180 days
Excluding PTP, prices data available only for 163 < 180 days
Excluding SWAPXI, prices data available only for 100 < 180 days
Excluding USDS, prices data available only for 7 < 180 days
Excluding MAG, prices data available only for 148 < 180 days
Excluding VTX, prices data available only for 93 < 180 days
Excluding IUSDS, prices data available only for 7 < 180 days
Excluding DIGITS, prices data available only for 111 < 180 days
Note: YAK, marketcap data available only for 179 < 180 days
Note: MAXI, marketcap data available only for 168 < 180 days
Excluding ACRE, prices data available only for 105 < 180 days
Excluding IME, prices data available only for 160 < 180 days
Excluding YETI, prices data available only for 45 < 180 days
Note: DCAU, marketcap data available only for 55 < 180 days
Note: ROCO, market

In [15]:
avalanche_tokens_data = avalanche_tokens_data[avalanche_tokens_data['symbol'].isin(list(map(str.lower, marketcaps.columns)))]

In [16]:
avalanche_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
14,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,26.910000,7310362250,15,1.939465e+10,9.585964e+08,28.890000,...,4.048230e+08,7.200000e+08,144.96000,-81.48437,2021-11-21T14:18:56.538Z,2.800000,858.22440,2020-12-31T13:15:21.540Z,None,2022-05-31T10:35:35.201Z
304,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.357717,97562173,305,1.790379e+08,6.483083e+06,0.381958,...,3.665385e+08,5.000000e+08,5.09000,-92.97671,2021-11-21T14:27:00.202Z,0.026588,1243.76436,2021-08-11T14:01:35.223Z,None,2022-05-31T10:36:51.348Z
472,wonderland,time,Wonderland,https://assets.coingecko.com/coins/images/1812...,50.040000,45365042,473,4.821136e+07,2.098220e+05,55.800000,...,1.216453e+07,9.567396e+05,10063.72000,-99.50015,2021-11-07T19:15:45.691Z,48.950000,2.76404,2022-05-31T09:44:40.055Z,None,2022-05-31T10:34:51.041Z
545,snowbank,sb,Snowbank,https://assets.coingecko.com/coins/images/1994...,220.240000,35114214,546,NaN,5.042700e+03,225.470000,...,1.596400e+05,NaN,8356.80000,-97.37463,2021-11-08T05:30:08.317Z,138.860000,57.99722,2022-01-23T06:18:28.046Z,None,2022-05-31T10:35:48.468Z
563,benqi,qi,BENQI,https://assets.coingecko.com/coins/images/1636...,0.014980,32907354,564,1.076677e+08,4.878122e+06,0.015734,...,7.200000e+09,7.200000e+09,0.39417,-96.21677,2021-08-24T03:58:11.390Z,0.008900,67.55289,2022-05-12T07:26:36.407Z,None,2022-05-31T10:37:06.593Z
680,avalaunch,xava,Avalaunch,https://assets.coingecko.com/coins/images/1546...,0.904038,22467804,681,9.017452e+07,1.688720e+06,0.963063,...,1.000000e+08,1.000000e+08,20.09000,-95.52361,2021-12-03T14:12:24.965Z,0.471748,90.60256,2021-07-21T05:06:03.647Z,None,2022-05-31T10:37:05.428Z
1042,crabada,cra,Crabada,https://assets.coingecko.com/coins/images/2001...,0.062328,7465109,1043,6.221782e+07,2.815210e+05,0.070301,...,1.000000e+09,1.000000e+09,2.96000,-97.89808,2021-11-22T17:18:31.018Z,0.057342,8.45196,2022-05-27T10:52:38.465Z,None,2022-05-31T10:35:46.212Z
1123,pangolin,png,Pangolin,https://assets.coingecko.com/coins/images/1402...,0.060206,6116850,1124,3.239066e+07,6.574250e+05,0.062965,...,5.380000e+08,5.380000e+08,18.85000,-99.68083,2021-02-19T15:34:15.118Z,0.049734,20.96856,2022-05-12T07:25:31.232Z,None,2022-05-31T10:37:05.746Z
1445,elk-finance,elk,Elk Finance,https://assets.coingecko.com/coins/images/1781...,0.478286,3132864,1446,NaN,1.246840e+05,0.498759,...,4.242424e+07,NaN,6.03000,-92.11516,2022-01-20T09:25:38.025Z,0.398954,19.13088,2022-05-12T07:19:50.553Z,None,2022-05-31T10:35:48.183Z
1450,yield-yak,yak,Yield Yak,https://assets.coingecko.com/coins/images/1765...,346.080000,3109903,1451,3.460046e+06,7.916300e+04,449.410000,...,1.000000e+04,1.000000e+04,16319.34000,-97.88344,2021-09-23T05:14:43.476Z,327.870000,5.35054,2022-05-12T16:16:14.771Z,None,2022-05-31T10:34:45.349Z


### Liquidity check

In [17]:
for token in avalanche_tokens.values():
    if 'wavax' in token['symbol']:
        print(token['id'])
        pprint(token)

wrapped-avax
id                                                wrapped-avax
symbol                                                   wavax
name                                              Wrapped AVAX
platforms    {'avalanche': '0xb31f66aa3c1e785363f0875a1b74e...
Name: 12919, dtype: object


In [18]:
base_token = avalanche_tokens['wrapped-avax']

In [19]:
base_token_address = base_token['platforms']['avalanche']

In [20]:
dex_subgraph_urls = {
    'Trader Joe': 'https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange',
    'Pangolin': 'https://api.thegraph.com/subgraphs/name/dasconnor/pangolin-dex'
} 

In [21]:
query = """
    query pairs ($token0: ID!, $token1: ID!) {
        pairs (where: {token0: $token0, token1: $token1}){
            id
            token0 {
              symbol
              id
            }
            token1 {
              symbol
              id
            }
            reserveUSD
        }
    }
    """

In [22]:
liquidities = {}
for id_ in avalanche_tokens_data['id']:
    token_addr = avalanche_tokens[id_]['platforms']['avalanche'].lower()
    symbol = avalanche_tokens[id_]['symbol'].upper()
    print(symbol, token_addr)
    
    token0, token1 = sorted([token_addr, base_token_address])
    row = {}
    for dex_name, url in dex_subgraph_urls.items():
        request = requests.post(url, json={'query': query, 'variables': {'token0': token0, 'token1': token1}})
        pairs = request.json()['data']['pairs']
        if len(pairs) == 0:
            print(f"There is no pair {symbol}-{base_token['symbol'].upper()} on {dex_name}")
            row[dex_name] = 0
            continue
        pair = pairs[0]
        row[dex_name] = float(pair['reserveUSD'])
    liquidities[symbol] = row
    
liquidities = pd.DataFrame.from_dict(liquidities, orient='index') 

AVAX fvweahmxkfeig8snevq42hc6whryy3efyavebmqdndgcgxn5z
There is no pair AVAX-WAVAX on Trader Joe
There is no pair AVAX-WAVAX on Pangolin
JOE 0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
TIME 0xb54f16fb19478766a268f172c9480f8da1a7c9c3
SB 0x7d1232b90d3f809a54eeaeebc639c62df8a8942f
QI 0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
XAVA 0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4
CRA 0xa32608e873f9ddef944b24798db69d80bbb4d1ed
PNG 0x60781c2586d68229fde47564546784ab3faca982
ELK 0xeeeeeb57642040be42185f49c52f7e9b38f8eeee
There is no pair ELK-WAVAX on Trader Joe
There is no pair ELK-WAVAX on Pangolin
YAK 0x59414b3089ce2af0010e7523dea7e2b35d776ec7
MAXI 0x7c08413cbf02202a1c13643db173f2694e0f73f0
DCAU 0x100cc3a819dd3e8573fd2e46d1e66ee866068f30
KLO 0xb27c8941a7df8958a1778c0259f76d1f8b711c35
ROCO 0xb2a85c5ecea99187a977ac34303b80acbddfa208


In [23]:
liquidities = liquidities.drop('AVAX')
liquidities.sort_values('Trader Joe', ascending=False)

,Trader Joe,Pangolin
JOE,1.442178e+07,4.118423e+04
XAVA,1.018696e+06,2.425626e+06
CRA,7.116701e+05,8.002346e+04
DCAU,3.176061e+05,2.116686e+03
QI,2.301092e+05,4.087513e+06
KLO,1.643028e+05,3.284831e+05
TIME,9.665478e+04,1.904564e+04
YAK,5.477894e+04,4.447625e+05
SB,9.277555e+03,2.300122e+01
PNG,6.799149e+03,2.119605e+06


In [24]:
liq_check = (liquidities > 5e6).any(axis=1).sort_values(ascending=False)
liq_check

JOE      True
TIME    False
SB      False
QI      False
XAVA    False
CRA     False
PNG     False
ELK     False
YAK     False
MAXI    False
DCAU    False
KLO     False
ROCO    False
dtype: bool

In [25]:
liq_check_failed = liq_check[~liq_check]
liq_check_failed

TIME    False
SB      False
QI      False
XAVA    False
CRA     False
PNG     False
ELK     False
YAK     False
MAXI    False
DCAU    False
KLO     False
ROCO    False
dtype: bool

In [26]:
liquidities = liquidities[liq_check.values]

In [27]:
liquidities

,Trader Joe,Pangolin
JOE,1.442178e+07,41184.23049


In [28]:
avalanche_tokens_data = avalanche_tokens_data[
    ~avalanche_tokens_data['symbol'].str.upper().isin(liq_check_failed.index)
]

In [29]:
avalanche_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
14,avalanche-2,avax,Avalanche,https://assets.coingecko.com/coins/images/1255...,26.910000,7310362250,15,1.939465e+10,958596385.0,28.890000,...,4.048230e+08,720000000.0,144.96,-81.48437,2021-11-21T14:18:56.538Z,2.800000,858.22440,2020-12-31T13:15:21.540Z,None,2022-05-31T10:35:35.201Z
304,joe,joe,JOE,https://assets.coingecko.com/coins/images/1756...,0.357717,97562173,305,1.790379e+08,6483083.0,0.381958,...,3.665385e+08,500000000.0,5.09,-92.97671,2021-11-21T14:27:00.202Z,0.026588,1243.76436,2021-08-11T14:01:35.223Z,None,2022-05-31T10:36:51.348Z


In [30]:
marketcaps = marketcaps[marketcaps.columns[marketcaps.columns.isin(avalanche_tokens_data['symbol'].str.upper())]]

In [31]:
prices_data = prices_data[prices_data.columns[prices_data.columns.isin(avalanche_tokens_data['symbol'].str.upper())]]

### Other checks

Nothing here

## Modeling

### Metrics description 

For performance investigation next three metrics are used:
1. Monthly returns
1. Tracking error for monthly returns compared to benchmark monthly returns
1. Weights standard deviation (root-mean-square deviation, RMSD) from target weights

Let $m$ be the number of assets and correspondent target weights are $w_1, w_2, \ldots , w_m$, where $\sum\limits_{i} w_i = 1$

#### Benchmark values calculation

For performance comparison, a benchmark portfolio is being fully rebalanced to the target weights at the first day of each month. Also there is a theoretical index, represented by a portfolio, whose weights are being adjusted to the target ones each day.

Let $p_i^{(k)}$ be the price of asset $i$ at the day $k$. Firstly assets daily returns are calculated: $$r_i^{(k)} = \frac{p_i^{(k)}}{p_i^{(k-1)}}-1$$

Then benchmark's values of each asset $i$ at the day $k$, namely $V_i^{(k)}$, and benchmark's total values $V^{(k)}$ are calculated by the next recurrent formulas:
$$V^{(1)} = 1, \qquad V_i^{(1)} = V^{(1)} \cdot w_i$$

$$V^{(k+1)} = \sum\limits_i V_i^{(k)}\cdot \left(1+r_i^{(k+1)}\right)$$ 

$$V_i^{(k+1)} = \begin{cases}
V^{(k+1)} \cdot w_i & \quad \text{if $k+1$ is the first day of month}\\
V_i^{(k)} \cdot \left(1+r_i^{(k+1)}\right) & \quad\text{otherwise}
\end{cases}$$

And values of the theoretical index are calculated by following:
$$V^{(1)} = 1, \qquad V_i^{(1)} = V^{(1)} \cdot w_i$$

$$V^{(k+1)} = \sum\limits_i V_i^{(k)}\cdot \left(1+r_i^{(k+1)}\right), \qquad V_i^{(k+1)} = V^{(k+1)} \cdot w_i$$ 

#### Metrics calculation

Given the sequence of portfolio $i$-th asset values $$V_i^{(1)}, V_i^{(2)}, \ldots, V_i^{(n)}$$ and the sequence of portfolio's total values $$V^{(1)}, V^{(2)}, \ldots, V^{(n)}$$ where $V^{(i)} = \sum\limits_{j=1}^m V_j^{(i)}$, mentioned above metrics are calculated in the next way:
$$Return_k = \frac{V^{(k)}}{V^{(k-1)}}-1$$

$$TrackingError_k = \sqrt{\frac1k \sum\limits_{i=1}^{k} (Return_i - ReturnBenchmark_i)^2 }$$

$$WeightsDeviation_k = \sqrt{\frac1m \sum\limits_{i=1}^m \left(w_i - \frac{V_i^{(k)}}{\sum\limits_{j}V_j^{(k)}}\right)^2}$$

Note that for monthly returns sequence of values at each 25-th day of month is used, and for weights deviation sequence of each day values is used

### Assets data

In [32]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [33]:
tickers = list(prices_data.columns)

In [34]:
fig = px.line(prices_data/prices_data.iloc[0]-1,
       labels={'variable': 'asset', 'value': 'return, %'})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    template='plotly_white',
    title='Assets return'
)
fig.show()

In [35]:
fig = px.line(prices_data.pct_change(),
       labels={'variable': 'asset', 'value': 'return, %'})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    template='plotly_white',
    title='Assets daily returns'
)
fig.show()

### Sqrt marketcap weights with limit 0.5

**Weighting requirements**

1. The maximum weight any one token can have is 50%.
1. All excess weight is proportionally redistributed to all uncapped tokens.

In [36]:
wlim = 0.5

In [37]:
target_weights = np.sqrt(marketcaps)
target_weights = target_weights.div(target_weights.sum(axis=1), axis=0)

In [38]:
overlim = target_weights[target_weights > wlim].sum(axis=1) - 0.5
under_weights = target_weights[target_weights <= wlim]
under_weights.fillna(0)
target_weights[target_weights <= wlim] += under_weights.div(under_weights.sum(axis=1), axis=0).mul(overlim, axis=0)
target_weights[target_weights > wlim] = 0.5

In [39]:
target_weights

,AVAX,JOE
date,,
2021-08-26,0.5,0.5
2021-08-27,0.5,0.5
2021-08-28,0.5,0.5
2021-08-29,0.5,0.5
2021-08-30,0.5,0.5
...,...,...
2022-05-26,0.5,0.5
2022-05-27,0.5,0.5
2022-05-28,0.5,0.5


In [40]:
fig = px.line(target_weights,
              labels={'value': 'weight, %', 'variable': ''},
              ## markers=True
             )
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights'
)

#### Benchmark

In [41]:
init_value = 1
cai_benchmark_values = pd.DataFrame(target_weights, index=prices_data.index) * init_value

In [42]:
assets_daily_returns = prices_data.pct_change()
assets_daily_returns.iloc[0] = 0

In [43]:
for i in range(1, len(prices_data.index)):
    cai_benchmark_values.iloc[i] = cai_benchmark_values.iloc[i-1] * (1 + assets_daily_returns.iloc[i])
    if cai_benchmark_values.index[i].day == 1:
        cai_benchmark_values.iloc[i] = cai_benchmark_values.iloc[i].sum() * target_weights.iloc[i]

#### Theoretical index

In [44]:
init_value = 1
cai_theoretical_values = pd.DataFrame(target_weights, index=prices_data.index) * init_value

In [45]:
assets_daily_returns = prices_data.pct_change()
assets_daily_returns.iloc[0] = 0

In [46]:
for i in range(1, len(prices_data.index)):
    cai_theoretical_values.iloc[i] = cai_theoretical_values.iloc[i-1] * (1 + assets_daily_returns.iloc[i])
    cai_theoretical_values.iloc[i] = cai_theoretical_values.iloc[i].sum() * target_weights.iloc[i]

#### Without management

In [47]:
init_value = 3e6
v_tokens = VTokens(assets=tickers)
cai = Index(v_tokens=v_tokens, assets=tickers,
            target_weights=target_weights.iloc[0], id_='cai')
cai.add_value(init_value, prices=dict(prices_data.iloc[0]))

cai_without_reb = []
for date, prices in prices_data.iterrows():
    cai_without_reb.append(cai.get_values(prices))
cai_without_reb = pd.DataFrame(cai_without_reb, index=prices_data.index)

#### Phuture rebalancing

In [48]:
engine = Engine(min_trade_value=100, max_price_impact_loss=0.03)

In [49]:
min_trade = 0.001
init_value = 3e6
v_tokens = VTokens(assets=tickers)
cai = Index(v_tokens=v_tokens, assets=tickers,
            target_weights=target_weights.iloc[0], id_='cai')
cai.add_value(init_value, prices=dict(prices_data.iloc[0]))

cai_values = []
rebalancing_hist = pd.DataFrame(False, index=prices_data.index, columns=['Trade execution', 'Order update']) 
for date, prices in prices_data.iterrows():
    cai.target_weights = target_weights.loc[date]
    if date.day == 1:
        cai.update_order(prices)
        rebalancing_hist['Order update'][date] = True
        print(date, "Reweight event at the contract side")
    engine.min_trade_value = min_trade * sum(cai.get_values(prices).values())
    trade = engine.run([cai])
    if trade:
        rebalancing_hist['Trade execution'][date] = True
        trade.execute(prices)
        print(date, trade)
    cai_values.append(cai.get_values(prices))
cai_values = pd.DataFrame(cai_values, index=prices_data.index)

2021-09-01 00:00:00 Reweight event at the contract side
2021-09-01 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=21297.30, value_before_scaling=21297.30, PI_loss=0.83%)
2021-10-01 00:00:00 Reweight event at the contract side
2021-10-01 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111680.75, value_before_scaling=698295.18, PI_loss=3.00%)
2021-10-02 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111680.09, value_before_scaling=586614.43, PI_loss=3.00%)
2021-10-03 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111679.37, value_before_scaling=474934.34, PI_loss=3.00%)
2021-10-04 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111678.56, value_before_scaling=363254.97, PI_loss=3.00%)
2021-10-05 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111677.55, value_before_scaling=251576.41, PI_loss=3.00%)
2021-10-06 00:00:00 Trade(assetSell=JOE, assetBuy=AVAX, value=111680.91, value_before_scaling=139898.86, PI_loss=3.00%)
2021-10-07 00:00:00 Trade(assetSell=JOE, assetBuy=

In [50]:
rebalancing_hist.sum(axis=0)

Trade execution    30
Order update        9
dtype: int64

In [51]:
plotting.index_metrics(index_name='cai', 
                       index_values=cai_values, 
                       benchmark_values=cai_benchmark_values,
                       target_weights=target_weights,
                       ## rebalancing_hist=rebalancing_hist,
                       values_theoretical=cai_theoretical_values,
                       values_without_reb=cai_without_reb)